In [2]:
# !pip install scine-molassembler
import numpy as np
import hpc_env
hpc_env.lmod.use("/usr/prog/cadd/modules")
hpc_env.lmod.load("molpipe")
hpc_env.lmod.load("OpenEye")
hpc_env.lmod.load("xtb")
import molpipe
# from rdkit.Chem.PandasTools import LoadSDF
import pandas as pd
from rdkit import Chem
# from ppqm import jupyter as ppqm_jupyter
import tempfile
import os
# import nglview as nv
# import scine_molassembler as masm
import subprocess
home = os.environ["HOME"]

In [4]:
molecules_table = pd.read_csv('/home/zhuji2q/Cu-Catalyzed-Ligands-Design/Data/Paper_Unsynthesized-Ligands.csv')
molecules_table.head()

,Name,Smiles_Code
0,L32,O=C(NN1C(C=CC=C2)=C2C3=C1C(C)=CC(C)=C3)C4=NC(O...
1,L33,O=C(NN1C(C(C)=CC=C2)=C2C3=C1C(C)=CC=C3)C4=NC(O...
2,L34,O=C(NN1C(C=CC=C2)=C2C3=C1C(OC)=CC=C3)C4=NC(O)=...
3,L35,O=C(NN1C(C=CC=C2)=C2C3=C1C(CC)=CC=C3)C4=NC(O)=...
4,L36,O=C(NN1C(C=CC=C2)=C2C3=C1C(Br)=CC=C3)C4=NC(O)=...


In [6]:
##generate canonical smiles for all interested ligands
cano_smi = []
for i in molecules_table.Smiles_Code:
    s = Chem.MolFromSmiles(i)
    new_i = Chem.MolToSmiles(s)
    cano_smi.append(new_i)

In [9]:
#find the interested pattern in the ligands molecule
cp_query = Chem.MolFromSmarts("NCC(=O)N")
for i in cano_smi:
    s = Chem.MolFromSmiles(i)
    m = Chem.MolToSmarts(s)
    q = Chem.MolFromSmarts(m)
    cp_matches_1 = q.GetSubstructMatches(cp_query)
    if len(cp_matches_1) > 0:
        print(cp_matches_1)
        continue
    else:
        print(i)

((24, 18, 16, 17, 15),)
((23, 17, 15, 16, 14),)
((23, 17, 15, 16, 14),)
((23, 17, 15, 16, 14),)
((23, 17, 1, 0, 2),)
((26, 20, 1, 0, 2),)
((23, 17, 1, 0, 2),)
((24, 18, 1, 0, 2),)
((22, 16, 14, 15, 13),)
((23, 17, 1, 0, 2),)
((28, 22, 20, 21, 19),)
((23, 17, 1, 0, 2),)
((19, 13, 11, 12, 10),)
((26, 16, 1, 0, 2),)
((6, 7, 8, 9, 10),)
((22, 5, 6, 7, 8),)
((6, 7, 8, 9, 10),)
((24, 5, 6, 7, 8),)
((24, 18, 16, 17, 15),)
((22, 16, 14, 15, 13),)
((28, 22, 20, 21, 19),)
((24, 18, 16, 17, 15),)
((22, 16, 14, 15, 13),)
((28, 22, 20, 21, 19),)
((24, 18, 16, 17, 15),)
((26, 20, 18, 19, 17),)


In [10]:
#find Nitrogen Atom
def is_N(atom):
    anum = atom.GetAtomicNum()
    return anum == 7

#Adding Copper atom to ligands to form a catalyst complex
def mol_transform(target_mol):
    cp_query = Chem.MolFromSmarts("NCC(=O)N")
    mol_1 = Chem.MolFromSmiles(target_mol)
    smarts = Chem.MolToSmarts(mol_1)
    mol_2 = Chem.MolFromSmarts(smarts)
    cp_matches = mol_2.GetSubstructMatches(cp_query)

    metal = [atom for atom in  mol_2.GetAtoms() if atom.GetAtomicNum() == 29]
    metal_idx = metal[0].GetIdx()
    print(metal[0])
    # metal_atom = target_mol.GetAtomWithIdx(metal_idx)
    metal[0].SetFormalCharge(2)
    # N_list = [x.GetIdx() for x in  metal[0].GetNeighbors()]
    print(cp_matches[0])
    # print(N_list)
    N1 =  mol_2.GetAtomWithIdx(cp_matches[0][0])
    N2 =  mol_2.GetAtomWithIdx(cp_matches[0][-1])
    # last_atom.SetFormalCharge(-1)
    # last_atom.SetNumExplicitHs(1)
    N1.SetFormalCharge(-1)
    N1.SetNumExplicitHs(0)
    N2.SetFormalCharge(-1)
    N2.SetNumExplicitHs(0)
    Chem.SanitizeMol(mol_2)
    Chem.Kekulize(mol_2, True)
    mol_2_bonded = Chem.RWMol(mol_2)
    metal_atom = mol_2_bonded.GetAtomWithIdx(metal_idx)
    mol_2_bonded.AddBond(metal_idx,cp_matches[0][0], Chem.BondType.SINGLE)
    mol_2_bonded.AddBond(metal_idx, cp_matches[0][-1], Chem.BondType.SINGLE)
    masm_smiles = Chem.MolToSmiles(mol_2_bonded)
    return masm_smiles

In [11]:
masm_list=[]
for k,i in enumerate(cano_smi):
    test_smile = i + '.[Cu]'
    masm_smiles = mol_transform(test_smile)
    masm_list.append(masm_smiles)
len(masm_list)

(24, 18, 16, 17, 15)
(23, 17, 15, 16, 14)
(23, 17, 15, 16, 14)
(23, 17, 15, 16, 14)
(23, 17, 1, 0, 2)
(26, 20, 1, 0, 2)
(23, 17, 1, 0, 2)
(24, 18, 1, 0, 2)
(22, 16, 14, 15, 13)
(23, 17, 1, 0, 2)
(28, 22, 20, 21, 19)
(23, 17, 1, 0, 2)
(19, 13, 11, 12, 10)
(26, 16, 1, 0, 2)
(6, 7, 8, 9, 10)
(22, 5, 6, 7, 8)
(6, 7, 8, 9, 10)
(24, 5, 6, 7, 8)
(24, 18, 16, 17, 15)
(22, 16, 14, 15, 13)
(28, 22, 20, 21, 19)
(24, 18, 16, 17, 15)
(22, 16, 14, 15, 13)
(28, 22, 20, 21, 19)
(24, 18, 16, 17, 15)
(26, 20, 18, 19, 17)


26

In [ ]:
# Generate sdf initial files for the ligands
names = molecules_table['Name']
f_list = ['initial']
while len(f_list) > 0:
    f_list =[]
    for k, i in enumerate(masm_list):
        try:
            masm_mol = masm.io.experimental.from_smiles(i)
            masm_conf = masm.dg.generate_random_conformation(masm_mol)
            with tempfile.NamedTemporaryFile(suffix=".mol") as f:
                masm.io.write(f.name, masm_mol, masm_conf)
                f.flush()
                f.seek(0)
                molblock = f.read().decode("utf-8")
            masm_raw_mol = Chem.MolFromMolBlock(molblock, sanitize=False)
        except:
            f_list.append('false')
            # print('false')
        with Chem.SDWriter(f"{home}/Cu-Catalyzed-ligands-Design/Molecular_Generation/SIOC_Ligands_Jan18_L53_2023/masm_conf_{names[k]}.sdf") as w:
            masm_raw_mol.SetProp("_Name",f"{names[k]}")
            w.write(masm_raw_mol)


In [ ]:
# list = [23,35,45,54,55,56,58,59,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79]
for i in names:
    path_1 = f"{home}/Cu-Catalyzed-ligands-Design/Molecular_Generation/SIOC_Ligands_Jan18_L53_2023/masm_conf_{i}.sdf"
    path_2 = f"{home}/Cu-Catalyzed-ligands-Design/Molecular_Generation/SIOC_Ligands_Jan18_L53_2023/masm_conf_xtb_{i}"
    path_3 = f"{home}/Cu-Catalyzed-ligands-Design/Molecular_Generation/SIOC_Ligands_Jan18_L53_2023/masm_conf_xtb_{i}.out"
    path_4 = f"{home}/Cu-Catalyzed-ligands-Design/Molecular_Generation/SIOC_Ligands_Jan18_L53_2023/masm_conf_xtb_{i}.error"
    !xtb {path_1} --opt --namespace {path_2} > {path_3} 2> {path_4}

In [ ]:
import logging
import os
from pathlib import Path

from functools import lru_cache, partial


open_utf8 = partial(open, encoding="utf-8")

def merge_sdf_files(dir: Path, output_file: Path):
    sdf_files = sorted(
        [dir + file for file in os.listdir(dir) if file.endswith(".xtbopt.sdf")])

    _logger.info(f"Found {len(sdf_files)} sdf files. Merging...")

    with open_utf8(output_file, "w") as out_sdf:
        for sdf_file in sdf_files:
            with open_utf8(sdf_file, "r") as sdf_in:
                out_sdf.write(sdf_in.read())
dir = f'{home}/Cu-Catalyzed-ligands-Design/Molecular_Generation/SIOC_Ligands_Jan18_L53_2023/'               
_logger = logging.getLogger(__name__)

In [ ]:
merge_sdf_files(dir, 'SIOC_ligands_Jan18_L53.sdf')

In [ ]:
import molpipe
suppl = Chem.SDMolSupplier('SIOC_ligands_Jan18_L53.sdf')
new_df = pd.DataFrame()
for mol in suppl:
    print(mol.GetProp("_Name"))
    cp_query = Chem.MolFromSmarts("[Cu]")
    # print(cp_query)
    cp_matches = mol.GetSubstructMatches(cp_query)
    print(cp_matches)
    rows_xtb = molpipe.pipe.get_all_properties_xtb(mol)
    pdf = pd.DataFrame(rows_xtb)
    pdf['Name'] = mol.GetProp("_Name")
    pdf['Cu_index'] = cp_matches[0][0]
    new_df = pd.concat([new_df,pdf])
    # pdf['Index']= str(vector)

In [ ]:
new_df

In [ ]:
new_df.to_csv('Add_metal_xtb_property.csv')

In [ ]:
new_df.columns

In [ ]:
list_p = ['h2o_covCN', 'h2o_alpha','h2o_mulliken_charges', 'h2o_f_plus', 'h2o_f_minus', 'h2o_f_zero']
for i in list_p:
    for k in range(len(new_df)):
        new_df[i].iloc[k] = new_df[i].iloc[k][int(new_df['Cu_index'].iloc[k])]

In [ ]:
new_df.to_csv('Add_metal_xtb_property_Cu.csv')

# Generate Steric Features: Buried Volume and BiteAngle

In [ ]:
from morfeus import BuriedVolume, BiteAngle,read_xyz


# directory = f"{home}/Ma_DaWei/Molecular_Generation/sdf_creation"
# x = 1
# # iterate over files in
# # that directory
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if f.endswith("xtbopt.sdf"):
#         masm_opt_mol = Chem.MolFromMolFile(f, sanitize=False)
#         index = get_numbers_from_filename(filename)
#         Chem.rdmolfiles.MolToXYZFile(masm_opt_mol,f"{home}/Ma_DaWei/Molecular_Generation/xyz_file/masm_opt_{index}.xyz")


##Check if the atom is metal atom based on Atomic Number
def is_metal(atom):
    anum = atom.GetAtomicNum()
    print(anum)
    return ((anum >= 3 and anum <=4) or
            (anum >= 11 and anum <= 13) or
            (anum >= 19 and anum <= 31) or
            (anum >= 37 and anum <= 50) or
            (anum >= 55 and anum <= 83) or
            (anum >= 87 and anum <= 103))


# Buriedvolume and biteangle calcualtion
def buried_vol_cal(f,idx):
    elements, coordinates = read_xyz(f)
    bv = BuriedVolume(elements, coordinates, idx) # idx, metal index
    return bv.fraction_buried_volume
def bite_angle_cal(f,idx,idx_list):
    elements, coordinates = read_xyz(f)
    bv = BiteAngle(coordinates, idx,idx_list[0],idx_list[1]) # idx_list, atom index connected to the metal center
    return bv.angle


# Show molecule with atom index
def mol_with_atom_index(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

In [ ]:
suppl = Chem.SDMolSupplier('SIOC_ligands_Jan18_L53.sdf')  # Read One Sdf containing all the molecules
name = []
# smiles = []
bv_list = []
ba_list = []
for mol in suppl:
    idx = 0
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() == 29: #check if the atom is Cu
            idx_list = [x.GetIdx() for x in atom.GetNeighbors()] # neighours index of metal center
            # print([x.GetIdx() for x in atom.GetNeighbors()])
            # print(idx)
        else:
            idx += 1 
    Chem.rdmolfiles.MolToXYZFile(mol,f"{home}/SIOC/Molecular_Generation/xyz_file/Temp.xyz") # Creating Temporary XYZ file for Morfeus Calculation
    bv = buried_vol_cal(f"{home}/SIOC/Molecular_Generation/xyz_file/Temp.xyz",idx)
    ba = bite_angle_cal(f"{home}/SIOC/Molecular_Generation/xyz_file/Temp.xyz",idx,idx_list)
    bv_list.append(bv)
    ba_list.append(ba)
        # if s == True:
        #     print(s)
    # metal_idx = metal[0][0]
    # metal_atom = mol.GetAtomWithIdx(metal_idx)
    # is_metal(metal_atom)
    name.append(mol.GetProp("_Name"))
    # smiles.append(Chem.MolToSmiles(mol)) 

In [ ]:
len(name)

In [ ]:
df = pd.DataFrame()
df['id'] = name
df['Buried_Vol'] = bv_list
df['Bite_Angle'] = ba_list
df.to_csv('SIOC_005_steric_des.csv', index= False)

In [ ]:
len(df)

In [ ]:
mol_with_atom_index(suppl[2])

In [ ]:
df.to_csv('Cu_Ligands.csv', index = False)